<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Terraform Cloud/Enterprise API Demo
Terraform Cloud (TFC) and Terraform Enterprise affor us three workflows:
* VCS-driven workflow
* CLI-driven workflow
* API-driven workflow

## Setup Environment Variables

In [ ]:
source ~/.tfapi

In [ ]:
echo "VAULT_TFE_KV: $VAULT_TFE_KV"

echo "TFE_TEAM_TOKEN: $TFE_TEAM_TOKEN"

echo "TFE_ORG_TOKEN: $TFE_ORG_TOKEN"

echo "TFE_ORG: $TFE_ORG"

echo "TFE_ADDR: $TFE_ADDR"

echo "VCS_OATH_TOKEN_ID: $VCS_OAUTH_TOKEN_ID"

In [ ]:
export VCS_WORKSPACE_NAME="vcs-driven-workspace"
export TF_VERSION="0.14.3"
export WORKING_DIRECTORY=""
export VCS_REPO="tio-hashicorp/tfe-demo-aws"
export VCS_BRANCH="" #  for default

---
## Workspace 


### Create Workspace with a VCS connection.  
This is useful for VCS-driven workflow.   
API Call: 
```
POST /organizations/:organization_name/workspaces
```

In [ ]:
cat <<EOF > /tmp/payload.json
{
  "data": {
    "attributes": {
      "name": "${VCS_WORKSPACE_NAME}",
      "terraform_version": "${TF_VERSION}",
      "working-directory": "${WORKING_DIRECTORY}",
      "vcs-repo": {
        "identifier": "${VCS_REPO}",
        "oauth-token-id": "${VCS_OAUTH_TOKEN_ID}",
        "branch": "${VCS_BRANCH}",
        "default-branch": true
      }
    },
    "type": "workspaces"
  }
}
EOF

cat /tmp/payload.json


In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_ORG_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/payload.json \
  ${TFE_ADDR}/api/v2/organizations/${TFE_ORG}/workspaces | jq -r .

### Create Workspace without specifying a VCS connection.  
This is useful when the TF runs are going to be initiated by a CI system.  
API Call: 
```
POST /organizations/:organization_name/workspaces
```

In [ ]:
export API_WORKSPACE_NAME="api-driven-workspace"

cat <<EOF > /tmp/payload.json
{
  "data": {
    "attributes": {
      "name": "${API_WORKSPACE_NAME}",
      "auto-apply": true
    },
    "type": "workspaces"
  }
}
EOF
cat /tmp/payload.json

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_ORG_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/payload.json \
  ${TFE_ADDR}/api/v2/organizations/${TFE_ORG}/workspaces | jq -r .

## List Workspaces
API Call: 
```
GET /organizations/:organization_name/workspaces
```

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_ORG_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  ${TFE_ADDR}/api/v2/organizations/${TFE_ORG}/workspaces | jq -r .

## Show workspace using name
API Call: 
```
GET /organizations/:organization_name/workspaces/:name
```

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_ORG_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  ${TFE_ADDR}/api/v2/organizations/${TFE_ORG}/workspaces/${VCS_WORKSPACE_NAME} | jq -r .

## Show workspace using id
API Call: 
```
GET /organizations/:organization_name/workspaces/:name
```


#### Get the  workspace ID

In [ ]:
export WORKSPACE_ID=$(curl -s \
  --header "Authorization: Bearer $TFE_ORG_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  ${TFE_ADDR}/api/v2/organizations/${TFE_ORG}/workspaces | \
  jq -r ".data[] | select (.attributes.name==\"${API_WORKSPACE_NAME}\") | .id")

In [ ]:
echo $WORKSPACE_ID

### Show workspace using workspace id

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_ORG_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  ${TFE_ADDR}/api/v2//workspaces/${WORKSPACE_ID} | jq -r .

---
## Workspace variables

In [ ]:
export VARIABLE_0_KEY="instance_type"
export VARIABLE_0_VALUE="t2.small"
export VARIABLE_0_DESCRIPTION="AWS Instance size"

### Create JSON payload, using aforementioned environment variables

In [ ]:
cat <<EOF > /tmp/payload.json
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"${VARIABLE_0_KEY}",
      "value":"${VARIABLE_0_VALUE}",
      "description":"${VARIABLE_0_DESCRIPTION}",
      "category":"terraform",
      "hcl":false,
      "sensitive":false
    }
  }
}
EOF
cat /tmp/payload.json

### Create variables in the workspace

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_TEAM_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/payload.json \
  ${TFE_ADDR}/api/v2/workspaces/${WORKSPACE_ID}/vars | jq -r .

### More variables

In [ ]:
export VARIABLE_1_KEY="aws_region"
export VARIABLE_1_VALUE="us-west-2"
export VARIABLE_1_DESCRIPTION="AWS Region"

In [ ]:
cat <<EOF > /tmp/payload.json
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"${VARIABLE_1_KEY}",
      "value":"${VARIABLE_1_VALUE}",
      "description":"${VARIABLE_1_DESCRIPTION}",
      "category":"terraform",
      "hcl":false,
      "sensitive":false
    }
  }
}
EOF
cat /tmp/payload.json

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_TEAM_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/payload.json \
  ${TFE_ADDR}/api/v2/workspaces/${WORKSPACE_ID}/vars | jq -r .

### Set Sensitive Variable

### Set AWS_ACCESS_KEY_ID

In [ ]:
export VARIABLE_2_KEY="AWS_ACCESS_KEY_ID"

In [ ]:
cat <<EOF > /tmp/payload.json
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"${VARIABLE_2_KEY}",
      "value":"${AWS_ACCESS_KEY_ID}",
      "category":"env",
      "hcl":false,
      "sensitive":true
    }
  }
}
EOF
cat /tmp/payload.json

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_TEAM_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/payload.json \
  ${TFE_ADDR}/api/v2/workspaces/${WORKSPACE_ID}/vars | jq -r .

In [ ]:
cat <<EOF > /tmp/payload.json
{
  "data": {
    "type":"vars",
    "attributes": {
      "key":"AWS_SECRET_ACCESS_KEY",
      "value":"${AWS_SECRET_ACCESS_KEY}",
      "category":"env",
      "hcl":false,
      "sensitive":true
    }
  }
}
EOF
cat /tmp/payload.json

### Set AWS_SECRET_ACCESS_KEY

In [ ]:
curl -s \
  --header "Authorization: Bearer $TFE_TEAM_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/payload.json \
  ${TFE_ADDR}/api/v2/workspaces/${WORKSPACE_ID}/vars | jq -r .

In [ ]:
vault write transform/template/nric \
  type=regex \
  pattern='[STFG]{1}(\d{7})[A-Z]{1}' \
  alphabet=builtin/numeric

In [ ]:
vault write transform/transformation/nric \
  type=fpe \
  template=nric \
  tweak_source=internal \
  allowed_roles=payments

In [ ]:
vault write transform/encode/payments value=T7180802G transformation=nric

---
## Create Terraform Configuration tar-gzip file

### Create a configuration version

In [ ]:
export CONTENT_DIRECTORY=./tf_config

export UPLOAD_FILE_NAME="./content-$(date +%s).tar.gz"

tar -zcvf "$UPLOAD_FILE_NAME" -C "$CONTENT_DIRECTORY" .

In [ ]:
cat <<EOF > /tmp/create_config_version.json
{
  "data": {
    "type": "configuration-versions"
  }
}
EOF

In [ ]:
export WORKSPACE_ID="ws-JQMwp4itmLo7zE4G"

UPLOAD_URL=($(curl -s \
  --header "Authorization: Bearer $TFE_TEAM_TOKEN" \
  --header "Content-Type: application/vnd.api+json" \
  --request POST \
  --data @/tmp/create_config_version.json \
  ${TFE_ADDR}/api/v2/workspaces/${WORKSPACE_ID}/configuration-versions \
  | jq -r '.data.attributes."upload-url"' ) )
  
echo $UPLOAD_URL

### Upload Configuration


In [ ]:
curl \
  --header "Content-Type: application/octet-stream" \
  --request PUT \
  --data-binary @"${UPLOAD_FILE_NAME}" \
  ${UPLOAD_URL}

**Note** Because we configured our workspace to auto-apply, we don't need to create a run. Otherwise, we could POST to the /runs API to create a run.

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">